In [1]:
%run lib/scrape_govuk.ipynb
metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')

import pyexcel
from io import BytesIO
import time

ods_files = [f for f in metadata['files'] if f['type'] == 'ODS']

out = Path('out')
out.mkdir(exist_ok=True)

def load_sheet(ods_file_info, sheet_name):
    display(f"Loading '{ods_file_info['title']}', sheet '{sheet_name}'...")
    start = time.time()
    ods_file = BytesIO(session.get(ods_file_info['url']).content)
    sheet = pyexcel.get_sheet(
        file_content=ods_file,
        file_type='ods',
        sheet_name=sheet_name,
        library='pyexcel-ods3'
    )
    display(f"in {int(time.time() - start)}s")
    return sheet

def load_book(ods_file_info):
    display(f"Loading '{ods_file_info['title']}'...")
    start = time.time()
    ods_file = BytesIO(session.get(ods_file_info['url']).content)
    book = pyexcel.get_book(
        file_content=ods_file,
        file_type='ods',
        library='pyexcel-ods3'
    )
    display(f"in {int(time.time() - start)}s")
    return book

for ods_file_info in ods_files:
    if ods_file_info['title'] == 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1':
        book = load_book(ods_file_info)
        data = book['vi_05']
        display(f"Processing '{ods_file_info['title']}', sheet 'vi_05'")
        %run "Entry clearance visas granted by country of nationality(Vi_05).ipynb"
        entry_1 = tidy.copy()
        data = book['vi_01_q']
        display(f"Processing '{ods_file_info['title']}', sheet 'v1_01_q'")
        %run "Entry clearance visa applications and resolution by category ( vi_01_q).ipynb"
        entry_2 = tidy.copy()
        entry_1.rename(columns={'Year': 'Period'}, inplace=True)
        entry_1.replace(
            {'Period': r'$([0-9]{4})^'}, {'Period': r'year/\1'}, regex=True, inplace=True
        )
        for col in ['Type', 'Broad category', 'Applicant type', 'Category', 'Resolution']:
            entry_1[col] = 'All'
        # Todo: "World" or "Not UK"?
        entry_2['Country of nationality'] = 'World'
        pd.concat([entry_1, entry_2], sort=True).to_csv(out / 'entry_visas.csv', index=False)
    elif ods_file_info['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 5':
        data = load_sheet(ods_file_info, 'as_22_q')
        display(f"Processing '{ods_file_info['title']}', sheet 'as_22_q'")
        %run "Arrivals under Dublin regulations(As_22_q).ipynb"
    elif ods_file_info['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 2 second edition':
        data = load_sheet(ods_file_info, 'as_04')
        display(f"Processing '{ods_file_info['title']}', sheet 'as_04'")
        %run "as_04 Asylum applications from main applicants and dependants.ipynb"
    elif ods_file_info['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 4':
        book = load_book(ods_file_info)
        data = book['as_16_q']
        display(f"Processing '{ods_file_info['title']}', sheet 'as_16_q'")
        %run "Asylum seekers receiving support(As_16_q).ipynb"
        data = book['as_19_q']
        %run "Refugees resettled(As_19_q).ipynb"
    elif ods_file_info['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 1 second edition':
        data = load_sheet(ods_file_info, 'as_01')
        display(f"Processing '{ods_file_info['title']}', sheet 'as_01'")
        %run "Home Office, Immigration Statistics October to December 2016, Asylum table as 01 q.ipynb"

"Loading 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1'..."

'in 19s'

"Processing 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1', sheet 'vi_05'"

,Geographical region,Country of nationality,Year,Value


['Virgin Islands (US)',
 'Burkina',
 'Faroe Islands',
 'Kiribati',
 'Anguilla (British)',
 'American Samoa',
 'Serbia and Montenegro',
 'America North',
 'Burma',
 'St. Helena (British)',
 'Cyprus',
 'St. Pierre and Miquelon',
 'Central African Republic',
 'Niue',
 'Bosnia and Herzegovina',
 'Guinea-Bissau',
 'Asia South East',
 'Micronesia',
 'Pitcairn Islands (British)',
 'EU 2',
 'Cook Islands',
 'Kosovo',
 'Palau',
 'Asia Central',
 'Bermuda (British)',
 'Montserrat (British)',
 'Samoa',
 'Cyprus (Northern part of)',
 'Cayman Islands (British)',
 'Korea (South)',
 'Sudan (South)',
 'Gibraltar (British)',
 'Asia South',
 'St. Lucia',
 'Cocos (Keeling) Islands',
 'Spain',
 'Korea (North)',
 'Ireland',
 'St. Maarten (Dutch Part)',
 'EU Other',
 'Macau',
 'Middle East',
 'Africa Sub-Saharan',
 'Other and unknown',
 'Africa North',
 'Curacao',
 'Cambodia',
 'Europe Other',
 'Surinam',
 'Asia East',
 'America Central and South',
 'Turks and Caicos Islands (British)',
 'Virgin Islands (Br

...

/home/alex/.virtualenvs/databaker/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


"Loading 'Asylum data tables immigration statistics October to December 2017 volume 5', sheet 'as_22_q'..."

'in 6s'

"Processing 'Asylum data tables immigration statistics October to December 2017 volume 5', sheet 'as_22_q'"

/home/alex/.virtualenvs/databaker/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
